# Tidy Data

### Learning Goals
- describe the concept of tidy data
- recognize whether a given dataset is tidy or messy
- articulate why tidy data is desirable
- reshape a dataset between a long and a wide format (both ways)
- utilize common tools to reshape data into tidy form


Statistician Hadley Wickham draws an analaogy for data to Leo Tolstoy's quote "Happy families are all alike; every unhappy family is unhappy in its own way."  Wickham's insight is that tidy datasets are all alike but every messy dataset is messy in its own way.

Tidy data sets are desirable because they provide a standard way to connect the structure of the dataset (e.g., columns, physical layout) to its meaning.  Tidy data follows three principles:

- Every column is a variable
- Every row is an observation
- Every cell is a single value.

Messy data refers to any other arrangement of a dataset.


The five most common problems with messy datasets are
- column headers are values, not variable names
- multiple variables are stored in one column
- variables are stored in both rows and columns
- multiple types of observational units are stored in the same table
- a single observational unit is stored in multiple tables.


The most common tools to address messiness are
- melting (changing wide format data to long format data)
- string splitting
- casting (changing long format data to wide format data)

Tidying a dataset puts it in the form described above (3 principles), and tidying is a subset of data cleaning.

### Task
Read Wickham's paper Tidy Data (https://vita.had.co.nz/papers/tidy-data.pdf).
- Compare Table 4 and Table 6 (income and religion).  Describe the change that was made to tidy the dataset.
- Compare Table 7 and Table 8 (Billborad top hits for 2000).  Describe the change that was made to tidy the dataset.  In your own words, what is melting?  
- Compare Table 9 and Table 10 (tuberculosis).  Describe the change that was made to tidy the dataset.  What is the difference between the molten data set and the tidy data presented in Table 10a and 10b, respectively?
- Compare Table 11 and Table 12 (weather in Mexico).  In Table 11, where are variables stored?  How should we ultimately deal with tmax and tmin?  When the data set is tidy, what does each row represent?
- Revisit Table 8 (Billboard top hits).  Notice that each artist and time are duplicated multiple times.  Compare this to Table 13.  What changed?

### Common Data Manipulation Actions
- filter (subsetting or removing observations based on some condition)
- transform (adding or modifying variables, e.g., log transforming a single variable or computing force from mass and acceleration variables)
- aggregate (collapsing multiple values into a single value, such as taking the mean, or summing the total count)
- sort (changing the order of observations, for example, sorting by date or sorting alphabetically)

## Activity
We will reproduce several of the tidying examples from Wickham's paper you just read.

### Pew Research Center Data

In [5]:
df = pd.read_csv("pew-raw.csv")
df

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k
0,Agnostic,27,34,60,81,76,137
1,Atheist,12,27,37,52,35,70
2,Buddhist,27,21,30,34,33,58
3,Catholic,418,617,732,670,638,1116
4,Dont know/refused,15,14,15,11,10,35
5,Evangelical Prot,575,869,1064,982,881,1486
6,Hindu,1,9,7,9,11,34
7,Historically Black Prot,228,244,236,238,197,223
8,Jehovahs Witness,20,27,24,24,21,30
9,Jewish,19,19,25,25,30,95


#### Melt
This dataset needs to have the income values not in the column headers, but as values in an income column.
To tidy the dataset, we need to **melt** it.  The pandas library has a built-in function (melt()) that "unpivots" a DataFrame from a wide format to a long format.

The melt() function creates a special format of the DataFrame where one or more columns work as identifier variables while all other columns, considered measured variables, are unpivoted to the row axis, leaving just two non-identifier columns, variable and value.


The syntax is the following:

pandas.melt(frame, id_vars=None, value_vars=None,
 var_name=None, value_name='value', col_level=None)
 
 where the parameters are
- frame : DataFrame
- id_vars (which can be a tuple, list, or ndarray) : Column(s) to use as identifier variables.
- value_vars: column(s) to unpivot. If not specified, uses all columns that are not set as id_vars.
- var_name: name to use for the ‘variable’ column. If None it uses frame.columns.name or ‘variable’.
- value_name: name to use for the ‘value’ column.
- col_level: if columns are a MultiIndex then use this level to melt.

This is a lot of information, so we will work through an example using melt() before addressing the data set in Wickham's paper to understand these parameters.


In [3]:
# Raw data in form of dictionary:
data = {"Person":["Alan","Berta","Charlie","Danielle"], #Name of Person
        "House":["A","B","A","C"],                      #Name of houses they live in
        "Age":[32,46,35,28],                            #Age of Person
        "Books":[100,30,20,40],                         #Number of books owned
        "Movies":[10,20,80,60]                          #Number of movie watched
        }

# Converting the raw data into pandas DataFrame:
data_wide = pd.DataFrame(data)

# Printing the pandas DataFrame:
data_wide

,Person,House,Age,Books,Movies
0,Alan,A,32,100,10
1,Berta,B,46,30,20
2,Charlie,A,35,20,80
3,Danielle,C,28,40,60


If we use the melt() function without any parameters, it will melt all columns.

In [4]:
# Melting the DataFrame from wide to long format:
# without specifying any parameters:

data_wide.melt()

,variable,value
0,Person,Alan
1,Person,Berta
2,Person,Charlie
3,Person,Danielle
4,House,A
5,House,B
6,House,A
7,House,C
8,Age,32
9,Age,46


Instead, if we select identifier variables Person and House, these columns will not be melted, but all other columns will be melted to form a single column.  Two new columns are created: the variable column stores the names of the melted columns and the value columns stores the actualy values of the melted columns.

In [5]:
# Melting the DataFrame from wide to long format:
# id_vars

data_wide.melt(id_vars=["Person","House"]) #Identifier columns

,Person,House,variable,value
0,Alan,A,Age,32
1,Berta,B,Age,46
2,Charlie,A,Age,35
3,Danielle,C,Age,28
4,Alan,A,Books,100
5,Berta,B,Books,30
6,Charlie,A,Books,20
7,Danielle,C,Books,40
8,Alan,A,Movies,10
9,Berta,B,Movies,20


We could also have specified the names of all the columns not to melt as value_vars if we are only interested in a subset of the data.  Compare the output of the code below to the output of the code above and notice what was retained. If instead we set value_vars to include Books, Movies, and Age, the dataframes of the example above and below would be equivalent.

In [6]:
#Melting the DataFrame from wide to long format:
#id_vars
#value_vars

data_wide.melt(id_vars=["Person"],             #Identifier columns
               value_vars=["Books","Movies"])  #Columns to be melted


,Person,variable,value
0,Alan,Books,100
1,Berta,Books,30
2,Charlie,Books,20
3,Danielle,Books,40
4,Alan,Movies,10
5,Berta,Movies,20
6,Charlie,Movies,80
7,Danielle,Movies,60


We might like to name the columns that default to variable and value with something more specific.  This is where the parameters var_name and value_name come in.

In [7]:
data_wide.melt(id_vars=["Person","House"],          #Identifier columns
               value_vars=["Age","Books","Movies"], #Columns to be melted
               var_name="Info",                     #Renaming the variable column name
               value_name="Numerical")              #Renaming the value column name


,Person,House,Info,Numerical
0,Alan,A,Age,32
1,Berta,B,Age,46
2,Charlie,A,Age,35
3,Danielle,C,Age,28
4,Alan,A,Books,100
5,Berta,B,Books,30
6,Charlie,A,Books,20
7,Danielle,C,Books,40
8,Alan,A,Movies,10
9,Berta,B,Movies,20


Let's return to the Pew Research Center data from Wickham's paper.  We will tidy the data using the parameters discussed above.  We also will sort the DataFrame alphabetically on the variable religion.

In [6]:
# MELT-- We will use the columns "religion" as the identifier variable, use the name "income" for the variable column
# and the name "value" for the value column which will count the frequency of occurences in the table.
formatted_df = pd.melt(df,["religion"], var_name="income", value_name="freq")
# sort alphabetically by religion
formatted_df = formatted_df.sort_values(by=["religion"])
formatted_df.head(10)

,religion,income,freq
0,Agnostic,<$10k,27
30,Agnostic,$30-40k,81
40,Agnostic,$40-50k,76
50,Agnostic,$50-75k,137
10,Agnostic,$10-20k,34
20,Agnostic,$20-30k,60
41,Atheist,$40-50k,35
21,Atheist,$20-30k,37
11,Atheist,$10-20k,27
31,Atheist,$30-40k,52


### Billboard Top 100 Dataset

In [37]:
df = pd.read_csv("billboard.csv", encoding="mac_latin2")
df.head(10)

,year,artist.inverted,track,time,genre,date.entered,date.peaked,x1st.week,x2nd.week,x3rd.week,...,x67th.week,x68th.week,x69th.week,x70th.week,x71st.week,x72nd.week,x73rd.week,x74th.week,x75th.week,x76th.week
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,78,63.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,15,8.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,71,48.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,41,23.0,18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,57,47.0,45.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2000,Janet,Doesn't Really Matter,4:17,Rock,2000-06-17,2000-08-26,59,52.0,43.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2000,Destiny's Child,Say My Name,4:31,Rock,1999-12-25,2000-03-18,83,83.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2000,"Iglesias, Enrique",Be With You,3:36,Latin,2000-04-01,2000-06-24,63,45.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2000,Sisqo,Incomplete,3:52,Rock,2000-06-24,2000-08-12,77,66.0,61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2000,Lonestar,Amazed,4:25,Country,1999-06-05,2000-03-04,81,54.0,44.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# Melting
# We will use the first 7 columns as the identifier variables, use the name "week" for the variable column
# and the name "rank" for the value column 
id_vars = ["year","artist.inverted","track","time","genre","date.entered","date.peaked"]
df = pd.melt(frame=df,id_vars=id_vars, var_name="week", value_name="rank")
df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,x1st.week,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,x1st.week,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,x1st.week,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,x1st.week,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,x1st.week,57.0


In [17]:
results = df.dtypes
results

year                int64
artist.inverted    object
track              object
time               object
genre              object
date.entered       object
date.peaked        object
week               string
rank               object
dtype: object

We need to extract the week number from the string in the week column.  We can use regular expression.

In [39]:
# Formatting 
df["week"] = df['week'].str.extract('(\d+)', expand=False).astype(int) # regex \d+ matches one or more digits in 
df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57.0


Remove rows with NaN.

In [41]:
# Cleaning out unnecessary rows
df = df.dropna()
df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78.0
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15.0
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71.0
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41.0
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57.0


Convert the rank column to integers.

In [42]:
df['rank'] = df['rank'].astype(int)
df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57


Create a date column.

In [44]:
# Create "date" columns
df['date'] = pd.to_datetime(df['date.entered']) + pd.to_timedelta(df['week'], unit='w') - pd.DateOffset(weeks=1)
df.head()

,year,artist.inverted,track,time,genre,date.entered,date.peaked,week,rank,date
0,2000,Destiny's Child,Independent Women Part I,3:38,Rock,2000-09-23,2000-11-18,1,78,2000-09-23
1,2000,Santana,"Maria, Maria",4:18,Rock,2000-02-12,2000-04-08,1,15,2000-02-12
2,2000,Savage Garden,I Knew I Loved You,4:07,Rock,1999-10-23,2000-01-29,1,71,1999-10-23
3,2000,Madonna,Music,3:45,Rock,2000-08-12,2000-09-16,1,41,2000-08-12
4,2000,"Aguilera, Christina",Come On Over Baby (All I Want Is You),3:38,Rock,2000-08-05,2000-10-14,1,57,2000-08-05


Keep only relevant columns and sort by year, within years by artist, etc.

In [46]:
df = df[["year", "artist.inverted", "track", "time", "genre", "week", "rank", "date"]]
df = df.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])
df.head()

,year,artist.inverted,track,time,genre,week,rank,date
246,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,1,87,2000-02-26
563,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,2,82,2000-03-04
880,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,3,72,2000-03-11
1197,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,4,77,2000-03-18
1514,2000,2 Pac,Baby Don't Cry (Keep Ya Head Up II),4:22,Rap,5,87,2000-03-25


In [47]:
# Assigning the tidy dataset to a variable for future usage
billboard = df


### Tuberculosis
Recall from the Wickham paper you read, that the column names indicate whether the group is male or female and their age range.  For example m1524 means a male between the ages of 15 and 24, inclusive.
Also, there is a distinction between zeros and missing values due to the data collection process, and this distinction is important.

In [53]:
df = pd.read_csv("tb-raw.csv")
df

,country,year,m014,m1524,m2534,m3544,m4554,m5564,m65,mu,f014
0,AD,2000,0.0,0.0,1.0,0.0,0,0,0.0,NaN,NaN
1,AE,2000,2.0,4.0,4.0,6.0,5,12,10.0,NaN,3.0
2,AF,2000,52.0,228.0,183.0,149.0,129,94,80.0,NaN,93.0
3,AG,2000,0.0,0.0,0.0,0.0,0,0,1.0,NaN,1.0
4,AL,2000,2.0,19.0,21.0,14.0,24,19,16.0,NaN,3.0
5,AM,2000,2.0,152.0,130.0,131.0,63,26,21.0,NaN,1.0
6,AN,2000,0.0,0.0,1.0,2.0,0,0,0.0,NaN,0.0
7,AO,2000,186.0,999.0,1003.0,912.0,482,312,194.0,NaN,247.0
8,AR,2000,97.0,278.0,594.0,402.0,419,368,330.0,NaN,121.0
9,AS,2000,NaN,NaN,NaN,NaN,1,1,NaN,NaN,NaN


In [54]:
df = pd.melt(df, id_vars=["country","year"], value_name="cases", var_name="sex_and_age")
df.head()

,country,year,sex_and_age,cases
0,AD,2000,m014,0.0
1,AE,2000,m014,2.0
2,AF,2000,m014,52.0
3,AG,2000,m014,0.0
4,AL,2000,m014,2.0


In [55]:
# Extract Sex, Age lower bound and Age upper bound group
tmp_df = df["sex_and_age"].str.extract("(\D)(\d+)(\d{2})", expand=False)   # regular expression 
tmp_df.head()

,0,1,2
0,m,0,14
1,m,0,14
2,m,0,14
3,m,0,14
4,m,0,14


In [56]:
# Name columns
tmp_df.columns = ["sex", "age_lower", "age_upper"]
tmp_df.head()

,sex,age_lower,age_upper
0,m,0,14
1,m,0,14
2,m,0,14
3,m,0,14
4,m,0,14


In [57]:
# Create `age`column based on `age_lower` and `age_upper`
tmp_df["age"] = tmp_df["age_lower"] + "-" + tmp_df["age_upper"]
tmp_df.head()

,sex,age_lower,age_upper,age
0,m,0,14,0-14
1,m,0,14,0-14
2,m,0,14,0-14
3,m,0,14,0-14
4,m,0,14,0-14


In [58]:
# Merge 
df = pd.concat([df, tmp_df], axis=1)
df.head()

,country,year,sex_and_age,cases,sex,age_lower,age_upper,age
0,AD,2000,m014,0.0,m,0,14,0-14
1,AE,2000,m014,2.0,m,0,14,0-14
2,AF,2000,m014,52.0,m,0,14,0-14
3,AG,2000,m014,0.0,m,0,14,0-14
4,AL,2000,m014,2.0,m,0,14,0-14


In [59]:
# Drop unnecessary columns and rows
df = df.drop(['sex_and_age',"age_lower","age_upper"], axis=1)
df = df.dropna()
df = df.sort_values(ascending=True,by=["country", "year", "sex", "age"])
df.head()

,country,year,cases,sex,age
0,AD,2000,0.0,m,0-14
10,AD,2000,0.0,m,15-24
20,AD,2000,1.0,m,25-34
30,AD,2000,0.0,m,35-44
40,AD,2000,0.0,m,45-54


### Weather in Mexico

In [66]:
df = pd.read_csv("weather-raw.csv")
df.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,d7,d8
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,NaN,NaN


In [67]:
# put in long form
df = pd.melt(df, id_vars=["id", "year","month","element"], var_name="day_raw")
df.head()

,id,year,month,element,day_raw,value
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


In [68]:
# Extracting day
df["day"] = df["day_raw"].str.extract("d(\d+)", expand=False)  
df["id"] = "MX17004"
df.head()

,id,year,month,element,day_raw,value,day
0,MX17004,2010,1,tmax,d1,NaN,1
1,MX17004,2010,1,tmin,d1,NaN,1
2,MX17004,2010,2,tmax,d1,NaN,1
3,MX17004,2010,2,tmin,d1,NaN,1
4,MX17004,2010,3,tmax,d1,NaN,1


In [69]:
# To numeric values
df[["year","month","day"]] = df[["year","month","day"]].apply(lambda x: pd.to_numeric(x, errors='ignore'))
df.head()

,id,year,month,element,day_raw,value,day
0,MX17004,2010,1,tmax,d1,NaN,1
1,MX17004,2010,1,tmin,d1,NaN,1
2,MX17004,2010,2,tmax,d1,NaN,1
3,MX17004,2010,2,tmin,d1,NaN,1
4,MX17004,2010,3,tmax,d1,NaN,1


In [71]:
import datetime
# Creating a date from the different columns
def create_date_from_year_month_day(row):
    return datetime.datetime(year=row["year"], month=int(row["month"]), day=row["day"])

df["date"] = df.apply(lambda row: create_date_from_year_month_day(row), axis=1)
df.head()

,id,year,month,element,day_raw,value,day,date
0,MX17004,2010,1,tmax,d1,NaN,1,2010-01-01
1,MX17004,2010,1,tmin,d1,NaN,1,2010-01-01
2,MX17004,2010,2,tmax,d1,NaN,1,2010-02-01
3,MX17004,2010,2,tmin,d1,NaN,1,2010-02-01
4,MX17004,2010,3,tmax,d1,NaN,1,2010-03-01


In [72]:
df = df.drop(['year',"month","day", "day_raw"], axis=1)
df = df.dropna()
df.head()

,id,element,value,date
12,MX17004,tmax,27.3,2010-02-02
13,MX17004,tmin,14.4,2010-02-02
22,MX17004,tmax,24.1,2010-02-03
23,MX17004,tmin,14.4,2010-02-03
44,MX17004,tmax,32.1,2010-03-05


In [73]:
# Unmelting column "element" (separate out the tmax and tmin entries)
df = df.pivot_table(index=["id","date"], columns="element", values="value")
df.reset_index(drop=False, inplace=True)
df

element,id,date,tmax,tmin
0,MX17004,2010-02-02,27.3,14.4
1,MX17004,2010-02-03,24.1,14.4
2,MX17004,2010-03-05,32.1,14.2


## Task: Women in the Workforce
The intent of Tidy Tuesday is to provide a safe and supportive forum for individuals practicing their wrangling and data visualization skills.  Data is posted each Monday along with a chart or article.  The community works in R, but the data sets can also be tidied in Python.  Data is available in this repo:
https://github.com/rfordatascience/tidytuesday

Naviagate to the dataset from March 5, 2019.  This data comes the Bureau of Labor Statistics and the Census Bureau about women in the workforce. There are historical data about women's earnings and employment status, as well as detailed information about specific occupation and earnings from 2013-2016.

Read the file jobs_gender.csv and store it into a Pandas DataFrame.  Remove all null values, and put the data in tidy form.



In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
df_raw = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2019/2019-03-05/jobs_gender.csv",
      sep=',', index_col=False, dtype='unicode')

df_raw

,year,occupation,major_category,minor_category,total_workers,workers_male,workers_female,percent_female,total_earnings,total_earnings_male,total_earnings_female,wage_percent_of_male
0,2013,Chief executives,"Management, Business, and Financial",Management,1024259,782400,241859,23.6,120254,126142,95921,76.04207956112951
1,2013,General and operations managers,"Management, Business, and Financial",Management,977284,681627,295657,30.3,73557,81041,60759,74.97316173294999
2,2013,Legislators,"Management, Business, and Financial",Management,14815,8375,6440,43.5,67155,71530,65325,91.32531804837132
3,2013,Advertising and promotions managers,"Management, Business, and Financial",Management,43015,17775,25240,58.7,61371,75190,55860,74.29179412155872
4,2013,Marketing and sales managers,"Management, Business, and Financial",Management,754514,440078,314436,41.7,78455,91998,65040,70.69718906932759
...,...,...,...,...,...,...,...,...,...,...,...,...
2083,2016,Pumping station operators,"Production, Transportation, and Material Moving",Material Moving,19540,18917,623,3.188331627430911,62096,62053,67083,NaN
2084,2016,Refuse and recyclable material collectors,"Production, Transportation, and Material Moving",Material Moving,65170,58851,6319,9.696179223569127,32367,34488,24268,NaN
2085,2016,Mine shuttle car operators,"Production, Transportation, and Material Moving",Material Moving,732,722,10,1.366120218579235,43866,43808,NaN,NaN
2086,2016,"Tank car, truck, and ship loaders","Production, Transportation, and Material Moving",Material Moving,4198,4155,43,1.024297284421153,44079,44487,NaN,NaN


Continue tidying tasks.

### References
- Wickham, Hadley.  Tidy data.  The Journal of Statistical Software, vol. 59, 2014.  https://vita.had.co.nz/papers/tidy-data.pdf
- Tidy Tuesday podcast: https://www.tidytuesday.com
- Python for Data Science Chapter 12 (Tidy Data): https://byuidatascience.github.io/python4ds/tidy-data.html
- https://www.christopheryee.org/blog/makeovermonday-women-in-the-workforce/
- https://www.jeannicholashould.com/tidy-data-in-python.html
- https://tomaugspurger.github.io/posts/modern-5-tidy/
- Towards AI Understanding Melt Tutorial: https://pub.towardsai.net/understanding-pandas-melt-pd-melt-362954f8c125